In [7]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [8]:
import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.width', 100)

movie = pd.read_csv("F:\movie recom\movie.csv\movie.csv")
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movie.shape

(27278, 3)

In [ ]:
rating = pd.read_csv("F:\movie recom\rating.csv\rating.csv")
rating.head()

In [ ]:
rating.shape

In [ ]:
rating["userId"].nunique()

In [ ]:
df = movie.merge(rating, how="left", on="movieId")
df.head(50)

In [ ]:
df.shape

In [ ]:
comment_counts = pd.DataFrame(df["title"].value_counts())
comment_counts

In [ ]:
rare_movies = comment_counts[comment_counts["title"] <= 1000].index
common_movies = df[~df["title"].isin(rare_movies)]

In [ ]:
common_movies.shape

In [ ]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df.head()

In [ ]:
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df.head()

In [ ]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched

In [ ]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()

In [ ]:
movies_watched_df.shape

In [ ]:
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count.head(50)

In [ ]:
perc = len(movies_watched) * 60 / 100
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
len(users_same_movies)

In [ ]:
final_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]
final_df.head()

In [ ]:
final_df.shape

In [ ]:
corr_df = final_df.T.corr().unstack().sort_values()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df[corr_df["user_id_1"] == random_user]

In [ ]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

In [ ]:
top_users.shape

In [ ]:
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings["userId"].unique()
top_users_ratings.head()

In [ ]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

In [ ]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df.head()

In [ ]:
recommendation_df[recommendation_df["weighted_rating"] > 3.5]
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)

In [ ]:
movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"]


In [ ]:
user = 108170
movie = pd.read_csv("F:\movie recom\movie.csv\movie.csv")
rating = pd.read_csv("F:\movie recom\rating.csv\rating.csv")

In [ ]:
movie_id = rating[(rating["userId"] == user) & (rating["rating"] == 5.0)].sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]

In [ ]:
movie_df = user_movie_df[movie[movie["movieId"] == movie_id]["title"].values[0]]

In [ ]:
user_movie_df.corrwith(movie_df).sort_values(ascending=False).head(10)

In [ ]:
def item_based_recommender(movie_name, user_movie_df):
    movie = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie).sort_values(ascending=False).head(10)

In [ ]:
movies_from_item_based = item_based_recommender(movie[movie["movieId"] == movie_id]["title"].values[0], user_movie_df)

In [ ]:
movies_from_item_based[1:6].index